# Einführung

## Über dieses Notebook
In diesem Notebook zeige ich euch die **Grundlagen der Python-Programmierung** mit Schwerpunkt auf maschinellem Lernen. Ich stelle wichtige Bibliotheken wie **Numpy**, **Pandas**, **Scikit-learn** und **Matplotlib** vor und zeige euch, wie ihr diese für die Analyse und Modellierung von Daten verwenden könnt. Am Ende dieses Tutorials solltet ihr mit den Python-Grundlagen vertraut sein und diese Bibliotheken verwenden können, um einfache Modelle für maschinelles Lernen zu erstellen.

----
# Machine Learning
Machine Learning ist ein Bereich der künstlichen Intelligenz, der es Computern ermöglicht, aus Daten zu lernen und Muster oder Zusammenhänge zu erkennen.

Im kommenden Abschnitt werden wir das Thema anhand eines Beispiels näher beleuchten. Konkret werden wir eine einfache Klassifikation mit Hilfe einer logistischen Regression durchführen.

## Einrichten der Umgebung
In diesem Abschnitt erfährst du, wie du notwendige Python-Bibliotheken wie numpy, pandas, scikit-learn und matplotlib usw. installierst und erhältst eine kurze Erklärung zu den Funktionen jeder Bibliothek.

In [ ]:
# Name der Bibliothek und Abküzung
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Hier werden nur bestimmte Funktion aus der Bibliothek sklearn importiert
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

# Hier wird der Download für den Datensatz vorbereitet
file_id = '1uujka4TJygnHSrai5rDmao8z9SeP_PsE'
download_link = f"https://drive.google.com/uc?id={file_id}"

## Den Datensatz verstehen
Hier geht es darum, einen geeigneten Datensatz auszuwählen, ihn mit pandas zu laden und die verschiedenen Merkmale und grundlegende Statistiken des Datensatzes zu verstehen.

In [ ]:
# Einen Datensatz mit Pandas laden
df = pd.read_csv(download_link)

In [ ]:
# Die ersten fünf Zeilen des Datensatzes anzeigen lassen
df.head()

In [ ]:
# Übersicht über den Datensatz und seine Eigenschaften anzeigen lassen
df.info()

Diese Übersicht bietet wertvolle Informationen:
*   Der Datensatz umfasst 891 Zeilen und 12 Spalten.
*   Die Spalten ["Name", "Sex", "Ticket", "Cabin", "Embarked"] haben den Datentyp 'object', der später in einen numerischen Datentyp umgewandelt werden muss.
*   In den Spalten ["Age", "Cabin", "Embarked"] sind Nullwerte vorhanden.

In [ ]:
# Die Funktion describe() erzeugt eine statistische Zusammenfassung des DataFrames 'df'
df.describe()

  - Der Datensatz enthält Informationen über 891 Passagiere.
  - Etwa __38,38% der Passagiere in diesem Datensatz haben überlebt__, wie der Durchschnittswert von 0,3838 zeigt.
  - Die __durchschnittliche Klasse (Pclass) der Passagiere liegt bei ungefähr 2,31__, was darauf hindeutet, dass die Mehrheit der Passagiere in der 2. und 3. Klasse waren.
  - Das __durchschnittliche Alter der Passagiere beträgt ungefähr 29,7 Jahre__. Der jüngste Passagier ist etwa 0,42 Jahre alt (wahrscheinlich einige Monate alt) und der älteste 80 Jahre.
  - Im Durchschnitt haben Passagiere __0,52 Geschwister oder Ehepartner__ bei sich. Die höchste beobachtete Anzahl liegt bei 8.
  - Die __meisten Passagiere reisten ohne Eltern oder Kinder__ (durchschnittlicher Parch-Wert beträgt etwa 0,38). Einige hatten jedoch bis zu 6 Eltern oder Kinder bei sich.
  - Der __durchschnittliche Fahrpreis für Passagiere beträgt etwa 32,20 USD__. Die Fahrpreise variieren stark, wobei 50% der Passagiere zwischen 7,91 und 31 USD zahlen. Der höchste beobachtete Fahrpreis beträgt 512,33 USD.

In [ ]:
# "Unique Values" einer bestimmten Spalte anzeigen lassen
print("Unique Values: ", df['Embarked'].unique())
print(df['Embarked'].value_counts()) # Ignoriert fehlende Werte

__Aufgabe:__ Finde heraus welche "Unique Values" die Spalte "Pclass" hat

----
## Exploratory Data Analysis (EDA)
Die explorative Datenanalyse spielt eine entscheidende Rolle im maschinellen Lernen, da sie dabei hilft, ein tieferes Verständnis der Daten zu gewinnen. Dieser Prozess kann mithilfe von Bibliotheken wie pandas, matplotlib und seaborn durchgeführt werden, um verschiedene Visualisierungen wie Histogramme, Boxplots und Streudiagramme zu erstellen, die dabei helfen, Muster, Ausreißer und Beziehungen in den Daten zu identifizieren und zu interpretieren.

### Histogramm: Welcher Anteil hat überlebt?

In [ ]:
# Erstellen einer Abbildung
plt.figure(figsize=(10, 5))
# Ein Countplot (Balkendiagramm) auf dem Unterpunkt zeichnen
sns.countplot(x='Survived', data=df)
# Titel für den Unterpunkt festlegen
plt.title('Überlebt')
# Das Diagramm anzeigen
plt.show()

**Aufgabe:** Erstelle ein ähnliches Histogram für die Variable "Sex"

### Histogramm: Für die Variablen Age und Fare, um zu wissen, wie die Verteilungen der Werte aussehen

In [ ]:
# Percentiles berechnen
q25_age = np.percentile(df.Age, 25)
q50_age = np.percentile(df.Age, 50)
q75_age = np.percentile(df.Age, 75)

# Verteilung plotten
plt.figure(figsize=(10, 5))
sns.histplot(df.Age, kde=True, bins=30)

# Vertikale Linien für Perzentile hinzufügen
plt.axvline(x=q25_age, color='red', linestyle='--', label='25. Perzentil')
plt.axvline(x=q50_age, color='green', linestyle='--', label='50. Perzentile (median)')
plt.axvline(x=q75_age, color='blue', linestyle='--', label='75. Perzentile')

plt.legend()
plt.grid()
plt.title('Verteilung der Variable "Age" mit Perzentilen')
plt.show()

**Aufgabe:** Erstelle ein ähnliches Histogram für die Variable "Fare"

Die Variable 'Fare' hat einen Ausreißer, den wir entfernen sollten


In [ ]:
# Finde den maximalen Wert in der Spalte 'Fare'
max_fare = df['Fare'].max()

# Entferne Zeilen, in denen 'Fare' diesen maximalen Wert hat
df = df[df['Fare'] != max_fare]

### Korrelation: Wie groß ist der Zusammenhang zwischen den Variablen im Datensatz

In [ ]:
# Heatmap erstellen
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
fig=plt.gcf()
fig.set_size_inches(8,6)
plt.show()

**Aufgabe:** Prüfe, ob starke Korrelationen zwischen den Variablen vorliegen (nahe an 1 oder -1)

----
## Datenbereinigung
In diesem Abschnitt kümmern wir uns um die Nullwerte und werden alle Spalten mit dem Datentyp 'object' in einen numerischen Datentyp umwandeln.

### Mit Nullwerten umgehen

In [ ]:
# Wiederholung: In welchen Spalten waren Nullwerte?
df.info()

- Age: 177 Nullwerte
- Cabin: 687 Nullwerte
- Embarked: 2 Nullwerte

In [ ]:
# Da in der Spalte 'Cabin' viele Werte fehlen, werden wir die gesamte Spalte aus dem Datensatz entfernen
df.drop('Cabin', axis=1, inplace=True) # (axis=1 entfernt Spalten)

In [ ]:
# Entferne alle Zeilen, mit einem Nullwert in der Spalte 'Age'
df.dropna(subset=['Age'], axis=0, inplace=True) # (axis=0 entfernt Zeilen)

**Aufgabe:** Entferne alle Zeilen, mit einem Nullwert in der Spalte 'Embarked'

### Feature Engineering
Feature Engineering ist ein Prozess im Machine Learning, bei dem aus vorhandenen Daten neue relevante Merkmale oder Attribute extrahiert werden. Beispielsweise kann es sinnvoll sein, die Werte des Attributs 'Alter' zu modifizieren. Kontinuierliche Werte wie beim 'Alter' können bei der Modellierung Herausforderungen darstellen. Deshalb könnte ein neues Attribut erstellt werden, das das Alter in verschiedene Altersgruppen einteilt.

In [ ]:
# Erstelle neues Attribut mit Altergruppen
df['Age_group'] = 0
df.loc[df['Age'] <= q25_age, 'Age_group'] = 0
df.loc[(df['Age'] > q25_age) & (df['Age'] <= q50_age), 'Age_group'] = 1
df.loc[(df['Age'] > q50_age) & (df['Age'] <= q75_age), 'Age_group'] = 2
df.loc[df['Age'] > q75_age, 'Age_group'] = 3

**Aufgabe:** Erstelle ein neues Attribut, in welchem die Werte aus 'Fare' kategorisch umgewandelt werden. Verwende dafür die berechneten Percentile aus der EDA (q25_fare, q50_fare und q75_fare). Entferne anschließend die Attribute 'Age' und 'Fare'.

### Strings in numerische Werte umwandeln
In Machine-Learning-Projekten ist es oft erforderlich, Strings in numerische Werte umzuwandeln, da viele Algorithmen nur mit numerischen Werten arbeiten können.

In [ ]:
df['Sex'].replace(['male','female'], [0, 1], inplace=True)

**Aufgabe:** Konvertiere die Werte der Spalte 'Embarked' in numerische Werte.

### Weitere unwichtigen Attribute entfernen

**Aufgabe:** Die Attribute 'Name', 'Ticket' und 'PassengerId' erscheinen irrelevant. Bitte entferne diese.

----
## Predictive Modeling
In diesem Abschnitt konzentrieren wir uns auf das prädiktive Modellieren. Dabei verwenden wir unser DataFrame 'df', um unsere Feature-Variablen 'X' und die Zielvariable 'y' zu erstellen, wobei 'y' die zu prognostizierende Spalte 'Sex' darstellt. Nachdem wir unsere Daten in Trainings- und Testsets unterteilt haben, trainieren wir ein LogisticRegression-Modell und evaluieren dessen Leistung anhand von Accuracy (Genauigkeit) und ROC AUC (Area under the curve).

In [ ]:
# Hier wird die Spalte 'Sex' aus dem DataFrame 'df' entfernt und der resultierende DataFrame in 'X' gespeichert.
# 'X' enthält somit alle Feature-Variablen, die für die Vorhersage verwendet werden sollen.
X = df.drop('Sex', axis=1)

# 'y' wird als die Zielvariable definiert und enthält die Werte der Spalte 'Sex' aus 'df'.
# Dies ist die Variable, die wir mit unserem Modell vorhersagen möchten.
y = df['Sex']

In [ ]:
# Mit Hilfe der 'train_test_split'-Funktion wird der Datensatz in Trainings- und Testdaten unterteilt.
# 'random_state=42' stellt sicher, dass die Aufteilung reproduzierbar ist (seed).
# 'stratify=y' sorgt dafür, dass die Aufteilung der Daten in Training und Test die gleiche Verteilung der Zielvariable 'Survived' beibehält.
xtrain, xtest, ytrain, ytest = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# Hier wird ein Modell mit der Klasse LogisticRegression erstellt.
# Die Funktion .fit() sorgt dann dafür, dass unser Modell trainiert wird.
model = LogisticRegression()
model.fit(xtrain, ytrain)

In [ ]:
# Die Genauigkeit (Accuracy) / ROC AUC des Modells auf dem Testdatensatz wird berechnet und ausgegeben.
print('Accuracy: ', accuracy_score(model.predict(xtest), ytest))
print('ROC AUC: ', roc_auc_score(ytest, model.predict_proba(xtest)[:, 1]))

**Aufgabe:** Wiederhole den Teil 'Predictive Modeling' mit dem Attribut 'Survived' als Zielvariable (y)